## Create CNN Model and Optimize it using Keras Tuner

Here’s a concise Markdown cell you can add to your notebook to explain **HyperParameters** and their relevance to your Keras Tuner CNN optimization workflow:



In [1]:
!pip install keras-tuner


   -------------------- ------------------- 1/2 [keras-tuner]
   -------------------- ------------------- 1/2 [keras-tuner]
   -------------------- ------------------- 1/2 [keras-tuner]
   ---------------------------------------- 2/2 [keras-tuner]



In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.20.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 2us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

C:\Users\kojus\AppData\Local\Temp\ipykernel_12648\556418634.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

c:\Users\kojus\anaconda3\envs\ML_DM\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 46m 46s]
val_accuracy: 0.9135000109672546

Best val_accuracy So Far: 0.9135000109672546
Total elapsed time: 01h 09m 09s


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

c:\Users\kojus\anaconda3\envs\ML_DM\lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [14]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 22, 22, 64)     │       102,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 30976)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 112)            │     3,469,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,573,658 (13.63 MB)

 Trainable params: 3,573,658 (13.63 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 327s 191ms/step - accuracy: 0.9531 - loss: 0.1272 - val_accuracy: 0.9135 - val_loss: 0.2623
Epoch 5/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 343s 203ms/step - accuracy: 0.9688 - loss: 0.0847 - val_accuracy: 0.9145 - val_loss: 0.3059
Epoch 6/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 343s 180ms/step - accuracy: 0.9792 - loss: 0.0584 - val_accuracy: 0.9147 - val_loss: 0.3534
Epoch 7/10
1071/1688 ━━━━━━━━━━━━━━━━━━━━ 1:54 185ms/step - accuracy: 0.9868 - loss: 0.0375

----

Here’s a concise Markdown cell you can add to your notebook to explain **HyperParameters** and their relevance to your Keras Tuner CNN optimization workflow:



In [ ]:
## Understanding HyperParameters in Keras Tuner

Keras Tuner uses the `HyperParameters` class to define and manage the search space for model optimization.  
When you build your model with a function like `build_model(hp)`, the `hp` argument lets you specify which hyperparameters to tune, such as the number of filters, kernel size, dense units, and learning rate.

**Why is this relevant?**  
- HyperParameters allow you to systematically explore different model architectures and training settings.
- You can use methods like `hp.Int`, `hp.Float`, `hp.Choice`, and `hp.Boolean` to define tunable values.
- Keras Tuner will automatically try different combinations to find the best-performing model.

**Example from this notebook:**
```python
filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16)
kernel_size=hp.Choice('conv_1_kernel', values=[3,5])
units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16)
learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3])
```

**Key HyperParameters methods:**
- `hp.Int(name, min_value, max_value, step)`: Integer range.
- `hp.Float(name, min_value, max_value, step, sampling)`: Float range.
- `hp.Choice(name, values)`: Pick from a list.
- `hp.Boolean(name)`: True/False.
- `hp.Fixed(name, value)`: Fixed value (not tunable).
- `hp.conditional_scope(parent_name, parent_values)`: Activate parameters only under certain conditions (for model selection).

**Tip:**  
Use `HyperParameters` to make your model flexible and tunable. This helps you automate the search for the best architecture and training settings for your data.

For more details, see the [Keras Tuner documentation](https://keras.io/keras_tuner/).